Many thanks to the kernel [Melanoma Hair Remove](https://www.kaggle.com/vatsalparsaniya/melanoma-hair-remove). I just want to make some suggestions to this code in order to decrease the execution time.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

import matplotlib.pyplot as plt

import cv2

path = "/kaggle/input/siim-isic-melanoma-classification/"

train_df = pd.read_csv(path + 'train.csv')

test_df = pd.read_csv(path + 'test.csv')

The original code in [Melanoma Hair Remove](https://www.kaggle.com/vatsalparsaniya/melanoma-hair-remove) is as follows:

In [ ]:
# https://www.kaggle.com/vatsalparsaniya/melanoma-hair-remove
def hair_remove(image):

    # convert image to grayScale
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # kernel for morphologyEx
    kernel = cv2.getStructuringElement(1,(17,17))
    
    # apply MORPH_BLACKHAT to grayScale image
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    
    # apply thresholding to blackhat
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    
    # inpaint with original image and threshold image
    final_image = cv2.inpaint(image,threshold,1,cv2.INPAINT_TELEA)
    
    return final_image

The main part that takes the major part of the execution time is actually the "inpaint" method. Therefore, if we can replace this part by some similar alternatives, we can accelerate the process.
Let's change it by some consecutive blurring steps at the hair pixels:


In [ ]:
def hair_remove2(image):
    
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #ax[1].imshow(grayScale)

    # kernel for morphologyEx
    kernel = cv2.getStructuringElement(1,(17,17))

    # apply MORPH_BLACKHAT to grayScale image
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    #ax[2].imshow(blackhat)

    # apply thresholding to blackhat
    _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    #ax[3].imshow(threshold)
    
    im1 = image.copy()
    im1[threshold>0] = int(255*0.7)
    im1 = cv2.blur(im1, (10,10))
    #ax[1].imshow(im1)

    for n in range(4):
        im2 = image.copy()
        im2[threshold>0] = im1[threshold>0]
        im2 = cv2.blur(im2, (10,10))
        #ax[n+2].imshow(im2)
        im1 = im2.copy()



    final_image = image.copy()
    final_image[threshold>0] = im2[threshold>0]
    #ax[5].imshow(final_image)
    
    return final_image

Let's get the execution times and the final images:

In [ ]:
images = ['ISIC_0078712','ISIC_0080817','ISIC_0082348','ISIC_0109869','ISIC_0155012','ISIC_0159568','ISIC_0164145',
          'ISIC_0194550','ISIC_0194914','ISIC_0202023'
          ,'ISIC_0015719','ISIC_0074268','ISIC_0075914','ISIC_0084395','ISIC_0085718','ISIC_0081956'
              ]
time1 = []
time2 = []

fig, ax = plt.subplots(int(len(images)/2), 6, figsize=(50,50))

for i,image_name in enumerate(images):

    image = cv2.imread(path + "jpeg/train/" + image_name + ".jpg")
    image = cv2.resize(image,(1024,1024))

    ax[int(i/2),0+3*np.mod(i,2)].imshow(cv2.cvtColor(cv2.resize(image,(1024,1024)), cv2.COLOR_BGR2RGB))
    ax[int(i/2),0+3*np.mod(i,2)].set_title(image_name, fontsize=30)

    t1 = time.time()
    im1 = hair_remove(image)
    t2 = time.time()
    tt1 = t2-t1
    time1.append(tt1)
    ax[int(i/2),1+3*np.mod(i,2)].imshow(cv2.cvtColor(im1, cv2.COLOR_BGR2RGB))
    ax[int(i/2),1+3*np.mod(i,2)].set_title('inpaint ' + str("{:.1f}".format(t2-t1 )) + " sec", fontsize=30)

    image = cv2.imread(path + "jpeg/train/" + image_name + ".jpg")
    image = cv2.resize(image,(1024,1024))
    
    t1 = time.time()
    im2 = hair_remove2(image)
    t2 = time.time()
    tt2 = t2-t1
    time2.append(tt2)
    ax[int(i/2),2+3*np.mod(i,2)].imshow(cv2.cvtColor(im2, cv2.COLOR_BGR2RGB))
    ax[int(i/2),2+3*np.mod(i,2)].set_title('blurring ' + str("{:.1f}".format(t2-t1 )) + " sec, " + 
                      str("{:.1f}".format(tt1/tt2)) + " times faster" , fontsize=30);


In [ ]:
print("Number of images: ",len(time1))
diff1 = np.array(time1[:-6])/np.array(time2[:-6])
print("Mean of ratio of process times for hairy images: ", diff1.mean())
print("std of ratio of process times for hairy images: ", diff1.std())
print("Number of hairy images that was processed slower: ", len(diff1[diff1<=1]))
diff2 = np.array(time1[-6:])/np.array(time2[-6:])
print("Mean of ratio of process times for non-hairy images: ", diff2.mean())
print("std of ratio of process times for non-hairy images: ",diff2.std())
print("Number of non-hairy images that was processed slower: ", len(diff2[diff2<=1]))
print("Time elapsed for the method with inprint: ", sum(time1))
print("Time elapsed for the method with blurring: ", sum(time2))

As we see, only in one image, the blurring method is slower than the inpaint method.
The total execution times:

In [ ]:
print("The total execution time for these 16 images in inpaint method: ", "{:.1f}".format(sum(time1)), "sec")

In [ ]:
print("The total execution time for these 16 images in blurring method: ", "{:.1f}".format(sum(time2)), "sec")